This is a temporary file to experiment with reading and writing midi files with microtonal information, later this will hopefully be turned into regular .py files in /adaptivetuning and corresponding tests

In [2]:
# created with musescore2 to contain microtonal information like halfsharp accidentals but apparently musescore
# does not export this information to midi
with open("midi_files/micro_test.mid", mode='rb') as midifile:
    midi_micro = midifile.read()

In [3]:
# with running status
with open("midi_files/cd.mid", mode='rb') as midifile:
    midi_cd = midifile.read()

In [6]:
# without running status
with open("midi_files/cd2.mid", mode='rb') as midifile:
    midi_cd2 = midifile.read()

In [4]:
with open("midi_files/BWV_0227.mid", mode='rb') as midifile:
    midi_bach = midifile.read()

In [ ]:
for byte in midi_cd:
    print(format(byte, '08b'), format(byte, '02x'), byte)

In [14]:
# see http://www.music.mcgill.ca/~ich/classes/mumt306/StandardMIDIfileformat.html
# and http://www.somascape.org/midi/tech/mfile.html

# event:
# delta time: 1___ 1___ 1___ 0___ or 0___
# event status: 1___
# event data: 0___ 0___ 0___

def read_chunk(midi, start_i):
    chunk_type = midi[start_i:start_i+4].decode(encoding="ASCII")
    chunk_length = int.from_bytes(midi[start_i+4:start_i+8], byteorder='big')
    next_i = start_i + 8 + chunk_length
    chunk_data = midi[start_i+8:next_i]
    return chunk_type, chunk_length, chunk_data, next_i

def read_variable_length_quantity(midi, start_i):
    t = 0
    next_i = -1
    for i in range(start_i, start_i + 4):
        t *= 0b10000000
        t += midi[i] & 0b01111111
        if midi[i] & 0b10000000 == 0: # if first bit is not set -> this is the last byte
            next_i = i + 1
            break
    if next_i == -1:
        raise ValueError("Variable length seems to use more than 4 bytes")
    return t, next_i

def read_event(midi, start_i, chunk_length, running_status):
    i = start_i
    if midi[i] & 0b10000000 == 0:
        status = running_status
    else:
        status = midi[i]
        i += 1
    #print('status: ', bin(status))
    # MIDI EVENTS
    if status >= 0b10000000 and status <= 0b11101111:
        midi_event_type = status // 0b00010000
        channel_nr = status & 0b00001111
        if midi_event_type == 0b1000: # Note Off
            key = midi[i]
            velocity = midi[i+1]
            print("Note Off, channel: ", channel_nr, "key: ", key, "velocity: ", velocity)
            i+=2
        elif midi_event_type == 0b1001: # Note On
            key = midi[i]
            velocity = midi[i+1]
            print("Note On, channel: ", channel_nr, "key: ", key, "velocity: ", velocity)
            i+=2
        elif midi_event_type == 0b1010: # Aftertouch
            key = midi[i]
            pressure = midi[i+1]
            print("Aftertouch, channel: ", channel_nr, "key: ", key, "pressure: ", pressure)
            i+=2
        elif midi_event_type == 0b1011: # Control Change
            control_nr = midi[i]
            value = midi[i+1]
            print("Control Change, channel: ", channel_nr, "control_nr: ", control_nr, "value: ", value)
            i+=2
        elif midi_event_type == 0b1100: # Program Change
            program = midi[i]
            print("Program Change, channel: ", channel_nr, "program: ", program)
            i+=1
        elif midi_event_type == 0b1101: # Channel Pressure
            pressure = midi[i]
            print("Channel Pressure, channel: ", channel_nr, "pressure: ", pressure)
            i+=1
        elif midi_event_type == 0b1110: # Pitch Wheel
            pitchchange = midi[i] * 0b10000000 + midi[i+1] - 0x2000
            print("Pitch Wheel, channel: ", channel_nr, "pitchchange: ", pitchchange)
            i+=2
        else:
            raise ValueError("Unknown midi event type")
    # TODO SYSEX EVENTS F0 und F7 
    # META EVENTS
    elif status == 0b11111111:
        meta_event_type = midi[i]
        i += 1
        meta_event_length, i = read_variable_length_quantity(midi, i)
        data = midi[i:i+meta_event_length]
        print("Meta Event, type:", meta_event_type, "length:", meta_event_length)
        i = i + meta_event_length
    else:
        raise ValueError("Unknown Midi event status")
    return i, status

def read_track_data(midi, chunk_length):
    i = 0
    running_status = 0
    while i < chunk_length:
        delta_t, i = read_variable_length_quantity(midi, i)
        print("delta time: ", delta_t)
        i, running_status = read_event(midi, i, chunk_length, running_status)
    

def midi_to_human(midi): # type(midi) = bytes
    i = 0
    while i < len(midi):
        chunk_type, chunk_length, chunk_data, i = read_chunk(midi, i)
        print("\n", chunk_type, chunk_length)
        
        if chunk_type == 'MThd':
            midi_format = int.from_bytes(chunk_data[0:2], byteorder='big')
            nr_tracks = int.from_bytes(chunk_data[2:4], byteorder='big')
            print('midi_format: ', midi_format, 'nr_tracks:', nr_tracks)
            if chunk_data[4] & 0b10000000 == 0: # first bit of <division> == 0 -> metrical timing
                # ticks per quarter-note
                ticks_per_quarternote = chunk_data[4] * 0b100000000 + chunk_data[5]
                print('ticks_per_quarternote: ', ticks_per_quarternote)
            else: # first bit of <division> == 1 -> timecode
                frames_per_second = abs(int.from_bytes(chunk_data[4:5], byteorder='big', signed=True))
                sub_frame_resolution = int.from_bytes(chunk_data[5:6], byteorder='big')
                print('frames_per_second: ', frames_per_second, 'sub_frame_resolution: ', sub_frame_resolution)
        elif chunk_type == 'MTrk':
            read_track_data(chunk_data, chunk_length)
        else:
            raise ValueError("Unknown chunk type")
        
        #if midi[i] // 0b00010000 == 0b1000:
        #    print('note off, channel: ', midi[i] % 0b00010000 + 1)
        #if midi[i] // 0b00010000 == 0b1001:
        #    print('note on, channel: ', midi[i] % 0b00010000 + 1)
    if i != len(midi):
        raise ValueError("File length does not match the stated length of the chunks.")
    

TODO 1: class midifile to represent a midi file consisting of track objects consting of
message objects
option to insert new messages at absolut times, add new track

TODO 2: read midi file to midifile object

TODO 3: write midifile object to midifile

TEST: add midi tuning messages to file, can I read them with max?

TODO 4: integrate: read midifile to midifile object, tune it, add tuning messages to midifile, write it to file

BONUS: Translate midifile with tuning messages to midi file with tuning messages split to multiple channels or tracks

In [17]:
midi_to_human(midi_micro)


 MThd 6
midi_format:  1 nr_tracks: 1
ticks_per_quarternote:  480

 MTrk 193
delta time:  0
Meta Event, type: 88 length: 4
delta time:  0
Meta Event, type: 89 length: 2
delta time:  0
Meta Event, type: 81 length: 3
delta time:  0
Control Change, channel:  0 control_nr:  121 value:  0
delta time:  0
Program Change, channel:  0 program:  0
delta time:  0
Control Change, channel:  0 control_nr:  7 value:  100
delta time:  0
Control Change, channel:  0 control_nr:  10 value:  64
delta time:  0
Control Change, channel:  0 control_nr:  91 value:  0
delta time:  0
Control Change, channel:  0 control_nr:  93 value:  0
delta time:  0
Meta Event, type: 33 length: 1
delta time:  0
Note On, channel:  0 key:  60 velocity:  80
delta time:  455
Note On, channel:  0 key:  60 velocity:  0
delta time:  25
Note On, channel:  0 key:  59 velocity:  80
delta time:  455
Note On, channel:  0 key:  59 velocity:  0
delta time:  25
Note On, channel:  0 key:  60 velocity:  80
delta time:  455
Note On, channel:  0

In [ ]:
midi_to_human(midi_bach)